Jake Ardoin semester project for CSC2730. 

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [19]:
import IPython
import matplotlib
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

pd.options.mode.use_inf_as_na = True
pd.options.mode.chained_assignment = None



In [3]:
#TA: CHANGE THIS FILE NAME TO THE LOCATION OF THE FILE YOU WISH TO READ FROM
df = pd.read_json('/Users/PCnew/Documents/Datasets/News_Category_Dataset.json',
                  lines=True)
df[0:5]

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [4]:
#removing unwanted attributes TA: You can skip this if you used
#the 20k.json file, thought just downloading the actual data
#would be much simpler

df1 = df.drop(["authors", "date", "link"], axis=1)
df1.replace('', np.nan, inplace=True)
df1 = df1.dropna(axis=0)

In [5]:
df1.shape

(105398, 3)

In [6]:
#my computer cant do 100k of these freaking things
#TA: Chance this slice number to the number of examples
#that you want to test for doing grading
df1=df1[0:20000]
df1.shape

(20000, 3)

In [7]:
#this is explained at the end, it is to get rid of a category with only 1 example
df1 = df1[df1.category != 'GOOD NEWS']

In [8]:
#now that I've removed the blank spaces I'm left with 105398 rows of data. 
#first im going to define a few data groups I need for later
#then i'll make Bag of WOrds out of the headline column + description column
#this will be done over the next handful of cells so examples can be printed of each
headline = df1["headline"]
description = df1["short_description"]
category = df1["category"]


y = category
print("Sample of y")
print(y[0:5])
y.shape

Sample of y
0            CRIME
1    ENTERTAINMENT
2    ENTERTAINMENT
3    ENTERTAINMENT
4    ENTERTAINMENT
Name: category, dtype: object


(19999,)

In [9]:
#here I combine the description and the title together into a single unit
X = headline + " " + description
print("Sample of X")
print(X[0:5])
X.shape

Sample of X
0    There Were 2 Mass Shootings In Texas Last Week...
1    Will Smith Joins Diplo And Nicky Jam For The 2...
2    Hugh Grant Marries For The First Time At Age 5...
3    Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4    Julianna Margulies Uses Donald Trump Poop Bags...
dtype: object


(19999,)

In [10]:
docs_train, docs_test, y_train, y_test = train_test_split(X,
                                        y, test_size = 0.25, random_state = 123)

In [11]:
#played with the variables to figure out the effects to decide 
#on which numbers to test in each variable of the grid_search
vect = TfidfVectorizer(min_df = 0, max_df = 0.6 )
docmatrix = vect.fit_transform(docs_train)
docmatrix.shape

(14999, 24423)

In [12]:
pipe = Pipeline([
    ('vect', TfidfVectorizer(min_df=10, max_df=0.1)),
    ('svc', LinearSVC(C=1))])

In [13]:
parameters = {
    'vect__min_df': [0, 10],
    'vect__max_df': [0.4, 0.3, 0.2],
    'svc__C': [2, 1, .1]
}
grid_search = GridSearchCV(pipe, parameters, cv=5, verbose=0)
grid_search.fit(docs_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.1, max_features=None, min_df=10,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__min_df': [0, 10], 'vect__max_df': [0.4, 0.3, 0.2], 'svc__C': [2, 1, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
#Lets see what grid search has decided our best params are:
print(grid_search.best_params_)

{'svc__C': 1, 'vect__max_df': 0.3, 'vect__min_df': 0}


In [16]:
model = grid_search.best_estimator_
y_predicted = model.predict(docs_test)
print(accuracy_score(y_test, y_predicted))

0.671


The results from every test are as follows. I used grid.scores_ because its a bit easier to read than cv_results_ 
I also included cv_results_ just for completeness. 

In [29]:
df_scores = pd.DataFrame(data = grid_search.cv_results_)
file = open("df_scores_semesterproject.pdf", "w")
file.write(str(df_scores))
file.close()

In [21]:
print(df_scores)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_svc__C  \
0        0.859183      0.075236         0.062490        0.000019            2   
1        0.744985      0.049347         0.056253        0.007645            2   
2        0.796692      0.013967         0.056231        0.007638            2   
3        0.712332      0.015930         0.049988        0.006249            2   
4        0.802926      0.044835         0.059367        0.006252            2   
5        0.706088      0.011681         0.059360        0.006259            2   
6        0.727951      0.012501         0.059370        0.006241            1   
7        0.627987      0.006243         0.049987        0.006236            1   
8        0.712335      0.012494         0.062481        0.000013            1   
9        0.612552      0.006160         0.056230        0.007649            1   
10       0.696709      0.007641         0.049995        0.006246            1   
11       0.615470      0.021

In [30]:
from pprint import pprint
pprint(grid_search.grid_scores_)

[mean: 0.65064, std: 0.00769, params: {'svc__C': 2, 'vect__max_df': 0.4, 'vect__min_df': 0},
 mean: 0.60924, std: 0.00688, params: {'svc__C': 2, 'vect__max_df': 0.4, 'vect__min_df': 10},
 mean: 0.64964, std: 0.00887, params: {'svc__C': 2, 'vect__max_df': 0.3, 'vect__min_df': 0},
 mean: 0.60797, std: 0.00693, params: {'svc__C': 2, 'vect__max_df': 0.3, 'vect__min_df': 10},
 mean: 0.64058, std: 0.00654, params: {'svc__C': 2, 'vect__max_df': 0.2, 'vect__min_df': 0},
 mean: 0.59844, std: 0.00799, params: {'svc__C': 2, 'vect__max_df': 0.2, 'vect__min_df': 10},
 mean: 0.65551, std: 0.00697, params: {'svc__C': 1, 'vect__max_df': 0.4, 'vect__min_df': 0},
 mean: 0.62664, std: 0.00764, params: {'svc__C': 1, 'vect__max_df': 0.4, 'vect__min_df': 10},
 mean: 0.65584, std: 0.00751, params: {'svc__C': 1, 'vect__max_df': 0.3, 'vect__min_df': 0},
 mean: 0.62651, std: 0.00754, params: {'svc__C': 1, 'vect__max_df': 0.3, 'vect__min_df': 10},
 mean: 0.65004, std: 0.00516, params: {'svc__C': 1, 'vect__max_df

C:\Users\PCnew\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


Below is a list of all the categories. I set this up to figure out which category only had 1 member and was gviing me an error. That is why above I removed the review pertaining to the category "GOOD NEWS". Since the error is fixed the code below is no longer serves a purpose but I left it just for completeness sake. GOOD NEWS won't appear below since I reran the cells, but it was there at first and this is how I figured out which one was screwing with my model.

In [31]:
unique = []

for x in y:
    if x not in unique:
        unique.append(x)


In [32]:
#this is the list of all the categories
unique

['CRIME',
 'ENTERTAINMENT',
 'WORLD NEWS',
 'IMPACT',
 'POLITICS',
 'WEIRD NEWS',
 'BLACK VOICES',
 'WOMEN',
 'COMEDY',
 'QUEER VOICES',
 'SPORTS',
 'BUSINESS',
 'TRAVEL',
 'MEDIA',
 'TECH',
 'RELIGION',
 'SCIENCE',
 'LATINO VOICES',
 'EDUCATION',
 'COLLEGE',
 'PARENTS',
 'ARTS & CULTURE',
 'STYLE',
 'GREEN',
 'TASTE',
 'HEALTHY LIVING',
 'THE WORLDPOST']

In [33]:
len(unique)

27

In [34]:
#creating a dict to hold counts
catd = {"test" :0}
for item in unique:
    catd.update( {str(item) : 0})
catd

{'test': 0,
 'CRIME': 0,
 'ENTERTAINMENT': 0,
 'WORLD NEWS': 0,
 'IMPACT': 0,
 'POLITICS': 0,
 'WEIRD NEWS': 0,
 'BLACK VOICES': 0,
 'WOMEN': 0,
 'COMEDY': 0,
 'QUEER VOICES': 0,
 'SPORTS': 0,
 'BUSINESS': 0,
 'TRAVEL': 0,
 'MEDIA': 0,
 'TECH': 0,
 'RELIGION': 0,
 'SCIENCE': 0,
 'LATINO VOICES': 0,
 'EDUCATION': 0,
 'COLLEGE': 0,
 'PARENTS': 0,
 'ARTS & CULTURE': 0,
 'STYLE': 0,
 'GREEN': 0,
 'TASTE': 0,
 'HEALTHY LIVING': 0,
 'THE WORLDPOST': 0}

In [35]:
for x in y:
    catd[x] += 1
catd

{'test': 0,
 'CRIME': 339,
 'ENTERTAINMENT': 3050,
 'WORLD NEWS': 1487,
 'IMPACT': 256,
 'POLITICS': 7007,
 'WEIRD NEWS': 385,
 'BLACK VOICES': 766,
 'WOMEN': 604,
 'COMEDY': 856,
 'QUEER VOICES': 1023,
 'SPORTS': 517,
 'BUSINESS': 240,
 'TRAVEL': 163,
 'MEDIA': 575,
 'TECH': 104,
 'RELIGION': 200,
 'SCIENCE': 74,
 'LATINO VOICES': 223,
 'EDUCATION': 133,
 'COLLEGE': 18,
 'PARENTS': 501,
 'ARTS & CULTURE': 106,
 'STYLE': 292,
 'GREEN': 290,
 'TASTE': 212,
 'HEALTHY LIVING': 546,
 'THE WORLDPOST': 32}